In [1]:

import pandas as pd

In [2]:
%pwd

'C:\\Users\\Hp'

In [ ]:
dataset = pd.read_csv("C:/Users/Hp/Downloads/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv")
print(dataset)

In [ ]:
newData = dataset.drop(columns=['Time','ID','Mdate','Day','Sensor_ID','Sensor_Name'])

In [ ]:
newData

In [ ]:
newData.rename({'Hourly_Counts':'Customer_visit'}, axis=1, inplace=True)

In [ ]:
newData['Date'] =  newData.Date_Time.str[:10]

In [ ]:
newData['Date'] = pd.to_datetime(newData.Date)

In [ ]:
newData 

In [ ]:
newData.set_index(newData.Date, inplace=True)

In [ ]:
newData

In [ ]:
newData = newData.drop(columns=['Date_Time','Year','Month','Date'])

In [ ]:
 time_series_data = newData

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib import rcParams

In [ ]:
rcParams['figure.figsize'] = 10,6

In [ ]:
plt.xlabel('Date')
plt.ylabel('Number of customer visit')
plt.plot( time_series_data)

In [ ]:
# Determing rolling statistics
rolmean =  time_series_data.rolling(window =12).mean()
rolstd =  time_series_data.rolling(window =12).std()

In [ ]:
print(rolmean , rolstd)

In [ ]:
orignal_data = plt.plot( time_series_data, color = 'blue', label = 'Original')
mean = plt.plot(rolmean, color = 'yellow', label = 'Rolling Mean')
standard_deviation = plt.plot(rolstd, color = 'black', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block = False)

In [ ]:
# performing Dicky Fuller test
from statsmodels.tsa.stattools import adfuller

print('Result of Dicky_ fuller test Method')
dftest = adfuller(time_series_data['Customer_visit'], autolag = 'AIC')

dfoutput = pd.Series(dftest[0:4], index = ['Test_statistic', 'p_value','#Lags used', 'Number of observation  used'])
for key,value in dftest[4].items():
    dfoutput['critical value (%s) '%key]= value
    
print(dfoutput)

In [ ]:
# Estimating Trend 
time_series_data_logscale = np.log(time_series_data)
plt.plot(time_series_data_logscale)


In [ ]:
datasetlogscaledifferenceaverage = time_series_data_logscale - movingAverage
datasetlogscaledifferenceaverage.head(12)

# Removing Nan value now
datasetlogscaledifferenceaverage.dropna(inplace = True)
datasetlogscaledifferenceaverage.head(10)


In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity( time_series):
    
    Moving_average = time_series.rolling(window =12).mean()
    MovingSTD = timeseries.rolling(window =12).std()
    
    #plot rolling stat..
    orig = plt.plot(time_series, color = 'blue', label = 'Orignal')
    mean = plt.plot(Moving_average, color = 'red', label = 'Rolling Mean')
    std = plt.plot(MovingSTD, color = 'black', label = 'Rolling Std')
    plt.legend(loc= 'best')
    plt.title('rolling mean and standard deviation')
    plt.show(block = False)
    
    #perform Dicky-Fuller test
    print("result of Dicky- Fuller test")
    dftest = adfuller(timeseries['Customer_visit'], autolag = 'AIC')
    dfoutput= pd.Series(dftest[0:4], index = ['Test_statistic', 'p_value','#Lags used', 'Number of observation  used'])
    for key,value in dftest[4].items():
        dfoutput['critical value (%s) '%key]= value
    print(dfoutput)
    
    

In [ ]:
test_stationarity(datasetlogscaledifferenceaverage)

In [ ]:
datasetlogscaleminusMovingExponentialDecayAverage = time_series_data_logscale - exponentialDecayWeightedAverage
test_stationarity(datasetlogscaleminusMovingExponentialDecayAverage)

In [ ]:
datasetLogDiffshifting = time_series_data_logscale - time_series_data_logscale.shift()
plt.plot(datasetLogDiffshifting)

In [ ]:
datasetLogDiffshifting.dropna(inplace = True)
test_stationarity(datasetLogDiffshifting)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(time_series_data_logscale)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid